# Week 3 - Segmenting and Clustering Neighbourhooods in Toronto

In [15]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Part 1 - Creating the pandas DataFrame

In [16]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
df.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [17]:
# Ignore cells without a borough assigned
df = df[df['Borough'] != 'Not assigned']
df.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [18]:
#C Group data by the postal Code
df_new = df.groupby('Postal Code', sort=False).agg(', '.join)
df_new.head(10)

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
M1B,Scarborough,"Malvern, Rouge"
M3B,North York,Don Mills
M4B,East York,"Parkview Hill, Woodbine Gardens"


In [19]:
#Replace cells with a borough but a NOt assigned neighborhood with the neighborhood same as the borough
df_new.loc[df_new['Neighborhood'] =='Not assigned', 'Neighborhood'] = df_new.loc[df_new['Neighborhood'] =='Not assigned', 'Borough']
df_new.reset_index(inplace=True)
df_new.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [20]:
df_new.shape

(103, 3)

## Part 2 - Adding the Latitudes and Longitude Coordinates

In [21]:
df_new['Latitude'] = None
df_new['Longitude'] = None
df_new.head(5)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,None,None
1,M4A,North York,Victoria Village,None,None
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",None,None
3,M6A,North York,"Lawrence Manor, Lawrence Heights",None,None
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",None,None


In [23]:
#!pip install geocoder
#import geocoder
#print('Done')

for i, pc in enumerate(df_new['Postal Code']):
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(pc))
        lat_lng_coords = g.latlng
    
    if lat_lng_coords:
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    
    df_new.loc[i, 'Latitude'] = latitude
    df_new.loc[i, 'Longitude'] = longitude

df_new.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7525,-79.3299
1,M4A,North York,Victoria Village,43.7306,-79.3131
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6551,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7233,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6625,-79.3919
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6626,-79.5283
6,M1B,Scarborough,"Malvern, Rouge",43.8114,-79.1966
7,M3B,North York,Don Mills,43.7492,-79.3619
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7072,-79.3119
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6574,-79.378


In [24]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="my-Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Visualize a map of Toronto with neighbourhoods superimposed

In [27]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Four square Credentials

In [28]:
CLIENT_ID = 'PDGX0B2DRB4TG0FDDVMCZC43OT5KTTDGFTASV15XW4KDXPCR' # your Foursquare ID
CLIENT_SECRET = 'DW0NUVCPOLF5CRJGDS0LLKFYM53FV3WRARIHSRLZS3SACX4J' # your Foursquare Secret
VERSION = '20201227' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PDGX0B2DRB4TG0FDDVMCZC43OT5KTTDGFTASV15XW4KDXPCR
CLIENT_SECRET:DW0NUVCPOLF5CRJGDS0LLKFYM53FV3WRARIHSRLZS3SACX4J


### Function to process all neighbourhoods and return nearby venues

In [31]:

import requests
LIMIT = 50
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
venues = getNearbyVenues(names=df_new['Neighborhood'], latitudes=df_new['Latitude'], longitudes=df_new['Longitude'])


In [ ]:
print(venues.shape)
venues.head

In [35]:
venues.groupby('Neighborhood', as_index=False).count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,14,14,14,14,14,14
1,"Alderwood, Long Branch",4,4,4,4,4,4
2,Bayview Village,5,5,5,5,5,5
3,"Bedford Park, Lawrence Manor East",21,21,21,21,21,21
4,Berczy Park,50,50,50,50,50,50
5,"Birch Cliff, Cliffside West",4,4,4,4,4,4
6,"Brockton, Parkdale Village, Exhibition Place",50,50,50,50,50,50
7,"Business reply mail Processing Centre, South C...",50,50,50,50,50,50
8,"CN Tower, King and Spadina, Railway Lands, Har...",50,50,50,50,50,50
9,Caledonia-Fairbanks,7,7,7,7,7,7


In [128]:
# Check unique categories

print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 245 uniques categories.


In [129]:
venues['Venue Category']

0                                           Park
1                              Food & Drink Shop
2                                           Park
3                                  Grocery Store
4                                         Bakery
5                                    Coffee Shop
6                                 Breakfast Spot
7                                 Breakfast Spot
8                                    Event Space
9                                    Yoga Studio
10                                           Spa
11                                   Coffee Shop
12                                   Coffee Shop
13                                   Coffee Shop
14                            Italian Restaurant
15                                    Restaurant
16                               Thai Restaurant
17                           Distribution Center
18                                           Pub
19                          Gym / Fitness Center
20                  

# Part 3 - Analysis

In [69]:
# one hot encoding
venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
#Adding the neighbourhood column into venues
venues_onehot['Neighborhood'] = venues['Neighborhood']
venues_onehot.head()
temp = list(venues_onehot.columns)

if 'Neighborhood' in temp:
    temp.remove('Neighborhood')
    
fixed_columns = ['Neighborhood'] + temp
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Neighborhood,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Newsagent,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1

In [70]:
# Examine dataframe size
venues_onehot.shape

(1814, 245)

In [71]:
venues_grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
venues_grouped.shape

(96, 245)

In [78]:
# Sort venues in descneding order function
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [119]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = venues_grouped['Neighborhood']

for ind in np.arange(venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Badminton Court,Skating Rink,Shopping Mall,Sushi Restaurant,Supermarket,Vietnamese Restaurant,Hong Kong Restaurant,Bubble Tea Shop,Department Store
1,"Alderwood, Long Branch",Convenience Store,Gym,Performing Arts Venue,Pub,Fast Food Restaurant,Farmers Market,Farm,Field,Dry Cleaner,Fish & Chips Shop
2,Bayview Village,Trail,Construction & Landscaping,Park,Dog Run,Cuban Restaurant,Eastern European Restaurant,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
3,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Comfort Food Restaurant,Indian Restaurant,Pharmacy,Pub,Restaurant,Café,Butcher
4,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Restaurant,Cheese Shop,Beer Bar,Farmers Market,Bakery,Creperie,Café


## Cluster Neighbourhoods

In [120]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = venues_grouped.drop('Neighborhood', axis = 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=67).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 2, 2, 2, 2, 2, 2, 1])

In [121]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_new

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(inplace=True)

toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7525,-79.3299,0,Food & Drink Shop,Park,Zoo Exhibit,Dry Cleaner,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,M4A,North York,Victoria Village,43.7306,-79.3131,0,Grocery Store,Park,Zoo Exhibit,Dry Cleaner,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6551,-79.3626,2,Coffee Shop,Breakfast Spot,Bakery,Playground,Theater,Electronics Store,Food Truck,Restaurant,Italian Restaurant,Spa
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7233,-79.4504,1,Clothing Store,Men's Store,Bookstore,Women's Store,American Restaurant,Cosmetics Shop,Restaurant,Food Court,Furniture / Home Store,Toy / Game Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6625,-79.3919,2,Coffee Shop,Sandwich Place,Bank,Burrito Place,Falafel Restaurant,Café,Fried Chicken Joint,Gastropub,Theater,Italian Restaurant
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6626,-79.5283,1,Pharmacy,Skating Rink,Park,Café,Shopping Mall,Grocery Store,Bank,Farm,Falafel Restaurant,Eastern European Restaurant
6,M1B,Scarborough,"Malvern, Rouge",43.8114,-79.1966,1,Zoo Exhibit,Fast Food Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Field,Farmers Market,Farm,Falafel Restaurant,Event Space
7,M3B,North York,Don Mills,43.7492,-79.3619,2,Coffee Shop,Bubble Tea Shop,Soccer Field,Smoke Shop,Park,Gym,Grocery Store,Supermarket,Burger Joint,Gas Station
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7072,-79.3119,1,Pizza Place,Pharmacy,Athletics & Sports,Flea Market,Breakfast Spot,Café,Rock Climbing Spot,Bank,Intersection,Gastropub
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6574,-79.378,2,Coffee Shop,Café,Clothing Store,Cosmetics Shop,Ramen Restaurant,Theater,Bookstore,Fast Food Restaurant,Steakhouse,Plaza


## Visualize Data

In [122]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

### Cluster 1

In [123]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Food & Drink Shop,Park,Zoo Exhibit,Dry Cleaner,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,North York,0,Grocery Store,Park,Zoo Exhibit,Dry Cleaner,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
27,North York,0,Park,Residential Building (Apartment / Condo),Zoo Exhibit,Dry Cleaner,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
35,East York,0,Park,Intersection,Zoo Exhibit,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
45,North York,0,Park,Zoo Exhibit,Dry Cleaner,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
49,North York,0,Bakery,Park,Basketball Court,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
61,Central Toronto,0,Bus Line,Park,Swim School,Zoo Exhibit,Eastern European Restaurant,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
68,Central Toronto,0,Park,Business Service,Zoo Exhibit,Eastern European Restaurant,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
69,West Toronto,0,Park,Residential Building (Apartment / Condo),Zoo Exhibit,Dry Cleaner,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
88,Etobicoke,0,Park,Yoga Studio,Convenience Store,Skating Rink,Grocery Store,Tennis Court,Ethiopian Restaurant,Eastern European Restaurant,Electronics Store,Elementary School


### Cluster 2

In [124]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,1,Clothing Store,Men's Store,Bookstore,Women's Store,American Restaurant,Cosmetics Shop,Restaurant,Food Court,Furniture / Home Store,Toy / Game Store
5,Etobicoke,1,Pharmacy,Skating Rink,Park,Café,Shopping Mall,Grocery Store,Bank,Farm,Falafel Restaurant,Eastern European Restaurant
6,Scarborough,1,Zoo Exhibit,Fast Food Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Field,Farmers Market,Farm,Falafel Restaurant,Event Space
8,East York,1,Pizza Place,Pharmacy,Athletics & Sports,Flea Market,Breakfast Spot,Café,Rock Climbing Spot,Bank,Intersection,Gastropub
10,North York,1,Pizza Place,Grocery Store,Pub,Fast Food Restaurant,Latin American Restaurant,Gas Station,Bank,Bakery,Italian Restaurant,Japanese Restaurant
11,Etobicoke,1,Pizza Place,Sandwich Place,Chinese Restaurant,Tea Room,Dry Cleaner,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
14,East York,1,Bus Line,Pharmacy,Grocery Store,Coffee Shop,Metro Station,Breakfast Spot,Middle Eastern Restaurant,Café,Road,Pet Store
16,York,1,Hockey Arena,Field,Grocery Store,Trail,Park,Zoo Exhibit,Ethiopian Restaurant,Eastern European Restaurant,Electronics Store,Elementary School
17,Etobicoke,1,Fish & Chips Shop,College Rec Center,Electronics Store,Shopping Mall,Grocery Store,Park,Carpet Store,Falafel Restaurant,Farm,Farmers Market
18,Scarborough,1,Construction & Landscaping,Tea Room,Gym / Fitness Center,Dry Cleaner,Park,Event Space,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant


### Cluster 3

In [125]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2,Coffee Shop,Breakfast Spot,Bakery,Playground,Theater,Electronics Store,Food Truck,Restaurant,Italian Restaurant,Spa
4,Downtown Toronto,2,Coffee Shop,Sandwich Place,Bank,Burrito Place,Falafel Restaurant,Café,Fried Chicken Joint,Gastropub,Theater,Italian Restaurant
7,North York,2,Coffee Shop,Bubble Tea Shop,Soccer Field,Smoke Shop,Park,Gym,Grocery Store,Supermarket,Burger Joint,Gas Station
9,Downtown Toronto,2,Coffee Shop,Café,Clothing Store,Cosmetics Shop,Ramen Restaurant,Theater,Bookstore,Fast Food Restaurant,Steakhouse,Plaza
13,North York,2,Coffee Shop,Bubble Tea Shop,Soccer Field,Smoke Shop,Park,Gym,Grocery Store,Supermarket,Burger Joint,Gas Station
15,Downtown Toronto,2,Cosmetics Shop,Coffee Shop,Café,Seafood Restaurant,Gastropub,Japanese Restaurant,Hotel,Theater,Park,American Restaurant
20,Downtown Toronto,2,Coffee Shop,Cocktail Bar,Seafood Restaurant,Restaurant,Cheese Shop,Beer Bar,Farmers Market,Bakery,Creperie,Café
22,Scarborough,2,Park,Business Service,Coffee Shop,Korean BBQ Restaurant,Zoo Exhibit,Electronics Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
23,East York,2,Sporting Goods Shop,Coffee Shop,Shopping Mall,Furniture / Home Store,Bank,Burger Joint,Fish & Chips Shop,Smoothie Shop,Breakfast Spot,Restaurant
24,Downtown Toronto,2,Coffee Shop,Clothing Store,Hotel,Middle Eastern Restaurant,Cosmetics Shop,Bubble Tea Shop,Plaza,Poke Place,Diner,Pizza Place


### Cluster 4

In [126]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,3,Bar,Zoo Exhibit,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm


### Cluster 5

In [127]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Scarborough,4,Trail,Zoo Exhibit,Dry Cleaner,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
